In [1]:
# CARGAMOS LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime

import boto3
import s3fs
import fastparquet
import awswrangler as wr
import os

import pyarrow.parquet as pq
import pyarrow as pa

import requests
import json
from io import BytesIO
from pandas.tseries.offsets import BDay

import math

In [3]:
# INICIALIZAMOS UNA SESION EN AWS
session = boto3.Session(
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
    region_name=os.environ['S3_REGION'])

s3_client = boto3.client('s3')

OBJETIVO DE ESTE NOTEBOOK:
- DESCARGAR DE DISTINTOS ORIGENES LOS DATOS DE LOS ACTIVOS DEL IBEX35
- COMPARAR LOS DATOS OBTENIDOS 

## CARGA DE COMPOSICIÓN HISTÓRICA DE IBEX35

In [6]:
# LEEMOS EL FICHERO CON LA COMPOSICIÓN HISTORICA
ibex_constituents = pd.read_excel('data_processing/Historical Composition IBEX35.xlsx')
ibex_constituents.head()

,Activo,Inclusion,Exclusion
0,ACE,1991-01-02,2003-06-02
1,ACX,1991-01-02,1991-07-01
2,ALB,1991-01-02,1991-07-01
3,ASL,1991-01-02,1994-01-03
4,BBV,1991-01-02,2000-01-31


In [7]:
# ibex_constituents['Exclusion'] -= BDay(1)

# ibex_constituents['Inclusion'] = ibex_constituents['Inclusion'].dt.date
# ibex_constituents['Exclusion'] = ibex_constituents['Exclusion'].dt.date

# OBTENEMOS EL DIA ACTUAL
today = datetime.date.today().strftime('%Y-%m-%d')

# COMPLETAMOS LOS DATOS NULOS EN EL FICHERO DE COMPOSICIÓN HISTORICA CON LA FECHA ACTUAL
ibex_constituents.fillna(today,inplace=True)

# DEFINIMOS UNA VARIABLE PARA TODOS LOS ACTIVOS
activos = ibex_constituents.Activo
ibex_constituents['Ticker'] = activos.apply(lambda x: x.split('_')[0])

unique_activos = list(set(activos))

### LEEMOS LOS DATOS DE UN FICHERO PICKLE

In [8]:
import pickle
with open('./Datos TFM/stock_data.pkl', 'rb') as f:
    data = pickle.load(f)

In [10]:
# REALIZAMOS LA CARGA DE TODOS LOS DATOS HISTORICOS DEL ARCHIVO Y LOS CARGAMOS EN UN BUCKET DE S3
bucket_name = 'stock-market-historical-data'
raw_folder = 'raw/pickle_data/'
s3 = boto3.resource('s3')
failed_loads = []

for t in data.keys():
    try:
        file_name = t +'.parquet'
        full_path = raw_folder + file_name
        stock_data = data[t]
        table = pa.Table.from_pandas(stock_data)
        buf = BytesIO()
        pq.write_table(table, buf)

        s3.Object(bucket_name, full_path).put(Body=buf.getvalue())
    except:
        failed_loads.append(ticker)
        continue

In [11]:
close_series = {ticker: df.close
                for ticker, df in data.items()}
stock_df = pd.DataFrame(close_series)

In [12]:
# REALIZAMOS LA DESCARGA DE LOS DATOS DE YFINANCE Y LOS CARGAMOS EN UN BUCKET DE S3
raw_folder = 'processed/pickle_data/'
s3 = boto3.resource('s3')

file_name = t +'.parquet'
full_path = raw_folder + file_name
table = pa.Table.from_pandas(stock_df)
buf = BytesIO()
pq.write_table(table, buf)

s3.Object(bucket_name, full_path).put(Body=buf.getvalue())


{'ResponseMetadata': {'RequestId': '5G8PAESG4QVWC36B',
  'HostId': 'xwf5sElhC+wq6ssjvy96ePxBE9/bptql5LW3QxgEIsguOwIC8EX9EYLP9E9hP4veUGS7sZUK0NdKBRTMib5pCg==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xwf5sElhC+wq6ssjvy96ePxBE9/bptql5LW3QxgEIsguOwIC8EX9EYLP9E9hP4veUGS7sZUK0NdKBRTMib5pCg==',
   'x-amz-request-id': '5G8PAESG4QVWC36B',
   'date': 'Wed, 09 Aug 2023 11:37:52 GMT',
   'x-amz-version-id': 'nJ3IYTEyE47YBA79IA4uLADmqkmkUzbp',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"9b2d56a45e6ba6152c8aa14196e3570d"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"9b2d56a45e6ba6152c8aa14196e3570d"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'nJ3IYTEyE47YBA79IA4uLADmqkmkUzbp'}

In [13]:
stock_df

,SAN,BTO_1,BKIA_0,BKIA,NTGY,TRE,ACX_0,ACX,FER,ACS,...,DRC,PRS_0,PRS_1,SCYR_0,TEM,CAR,AGS,COL_0,A3TV,SGC
2003-01-02,2.251615,NaN,NaN,NaN,7.363463,NaN,4.761374,NaN,NaN,4.815096,...,16.50,49.590521,NaN,4.732566,5.88,NaN,NaN,NaN,NaN,9.35
2003-01-03,2.166648,NaN,NaN,NaN,7.270788,NaN,4.865828,NaN,NaN,4.815096,...,16.84,48.960800,NaN,4.740697,5.87,NaN,NaN,NaN,NaN,9.36
2003-01-07,2.251615,NaN,NaN,NaN,7.388738,NaN,4.778783,NaN,NaN,4.815096,...,16.29,48.488510,NaN,4.667513,6.03,NaN,NaN,NaN,NaN,9.19
2003-01-08,2.251615,NaN,NaN,NaN,7.582514,NaN,4.717851,NaN,NaN,4.905021,...,16.31,48.016219,NaN,4.651250,6.02,NaN,NaN,NaN,NaN,8.83
2003-01-09,2.251615,NaN,NaN,NaN,7.641489,NaN,4.752669,NaN,NaN,4.913196,...,16.51,47.543928,NaN,4.708171,6.13,NaN,NaN,NaN,NaN,9.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-17,1.890800,NaN,NaN,1.1240,16.160000,NaN,NaN,7.466,22.61,21.160000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-18,1.877000,NaN,NaN,1.1125,16.075000,NaN,NaN,7.318,22.83,21.150000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-19,1.934200,NaN,NaN,1.1260,16.300000,NaN,NaN,7.280,23.13,21.710000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-08-20,1.853800,NaN,NaN,1.0920,16.335000,NaN,NaN,6.950,22.75,21.340000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### LEEMOS LOS DATOS DE FICHEROS

In [14]:
ibex_file_data = pd.read_parquet('./Datos TFM/IBEX.parquet')
ibex_file_data = ibex_file_data.loc['2020-08-24':,:'VIS_0']
ibex_file_data

,ABE,ABG,ABG.P_0,ABG.P_1,ACS,ACX,ACX_0,AENA,ALM,AMS,...,SCYR_1,SGRE,SGRE_0,SLR,TEF,TL5,TRE,UNI,VIS,VIS_0
2020-08-24,NaN,NaN,NaN,NaN,18.598827,5.784362,NaN,122.00,9.059603,45.25,...,NaN,22.80,NaN,NaN,2.886744,NaN,NaN,NaN,61.697101,NaN
2020-08-25,NaN,NaN,NaN,NaN,18.092966,5.664651,NaN,122.50,8.988342,47.54,...,NaN,22.91,NaN,NaN,2.830378,NaN,NaN,NaN,61.162295,NaN
2020-08-26,NaN,NaN,NaN,NaN,17.974931,5.796000,NaN,124.20,8.874325,48.12,...,NaN,23.26,NaN,NaN,2.792532,NaN,NaN,NaN,61.599863,NaN
2020-08-27,NaN,NaN,NaN,NaN,17.915914,5.686265,NaN,124.20,8.793563,48.57,...,NaN,22.20,NaN,NaN,2.765959,NaN,NaN,NaN,61.259533,NaN
2020-08-28,NaN,NaN,NaN,NaN,17.890621,5.774386,NaN,128.80,8.812566,49.03,...,NaN,22.27,NaN,NaN,2.729724,NaN,NaN,NaN,60.384396,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-17,NaN,NaN,NaN,NaN,28.160000,8.982000,NaN,142.40,NaN,58.32,...,NaN,NaN,NaN,16.885,3.702000,NaN,NaN,0.9875,NaN,NaN
2023-03-20,NaN,NaN,NaN,NaN,28.570000,9.140000,NaN,141.95,NaN,58.26,...,NaN,NaN,NaN,16.620,3.754000,NaN,NaN,0.9965,NaN,NaN
2023-03-21,NaN,NaN,NaN,NaN,29.120000,9.320000,NaN,146.55,NaN,59.26,...,NaN,NaN,NaN,16.525,3.840000,NaN,NaN,1.0430,NaN,NaN
2023-03-22,NaN,NaN,NaN,NaN,28.990000,9.262000,NaN,145.50,NaN,58.74,...,NaN,NaN,NaN,16.255,3.832000,NaN,NaN,1.0200,NaN,NaN


In [15]:
ibex_file_data.isnull().sum(), ibex_file_data.shape[0]

(ABE        666
 ABG        666
 ABG.P_0    666
 ABG.P_1    666
 ACS          0
           ... 
 TL5        666
 TRE        666
 UNI        638
 VIS        325
 VIS_0      666
 Length: 68, dtype: int64,
 666)

In [16]:
complete_ibex_data = pd.concat([stock_df,ibex_file_data],axis=0)

ibex_historical_data = {}
processed_folder = 'processed/bme-data/'
file_name = 'ibex_historical_data'
full_path = processed_folder + file_name

table = pa.Table.from_pandas(complete_ibex_data)
buf = BytesIO()
pq.write_table(table, buf)

s3.Object(bucket_name, full_path).put(Body=buf.getvalue())

{'ResponseMetadata': {'RequestId': 'QBA6ZHYABY1GHTCZ',
  'HostId': 'LXMY//Wkcl9qbT1VaQ6IignJG8WkEdKrHSg/Vl+tvMnM+cgLKY+4fgZinbx0wpQmfnL+wVUe2wI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'LXMY//Wkcl9qbT1VaQ6IignJG8WkEdKrHSg/Vl+tvMnM+cgLKY+4fgZinbx0wpQmfnL+wVUe2wI=',
   'x-amz-request-id': 'QBA6ZHYABY1GHTCZ',
   'date': 'Wed, 09 Aug 2023 11:38:25 GMT',
   'x-amz-version-id': 'NK2FB1ewxZgodUdxTCz5vQ0tyrFA3Sh9',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4ebe40679404e46526a452fe125045dc"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"4ebe40679404e46526a452fe125045dc"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'NK2FB1ewxZgodUdxTCz5vQ0tyrFA3Sh9'}